In [3]:
import random
import numpy as np
import pandas as pd
from pathlib import Path

In [12]:
aapl_df = pd.read_csv(
    "aapl.csv",
    index_col="date",
    parse_dates=True,
    infer_datetime_format=True)

# Review the DataFrame
aapl_df.head()

,close,volume,open,high,low
date,,,,,
2014-09-22,101.06,52421660,101.80,102.14,100.58
2014-09-23,102.64,63255860,100.60,102.94,100.54
2014-09-24,101.75,59974260,102.16,102.85,101.20
2014-09-25,97.87,99689300,100.51,100.71,97.72
2014-09-26,100.75,62276770,98.53,100.75,98.40


In [ ]:
portret_df = aapl_df['close'].pct_change()

In [88]:
def perf_eval(portret_df):
    # Create a list for the column name
    columns = ["Backtest Results"]

    # Create a list holding the names of the new evaluation metrics
    metrics = [
        "Annualized Return",
        "Cumulative Returns",
        "Annual Volatility",
        "Sharpe Ratio",
        "Sortino Ratio",
        "Best Winning Streak",
        "Worst Losing Streak"]

    # Initialize the DataFrame with index set to the evaluation metrics and the column
    portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)

    # Calculate annualized return
    portfolio_evaluation_df.loc["Annualized Return"] = round(
        portret_df.mean() * 252
        ,2
        )

    # Calculate cumulative return

    portfolio_evaluation_df.loc["Cumulative Returns"] = round(
        ((1 + portret_df.cumsum()) -1)[-1]
        ,2
        )
        

    # Calculate annual volatility
    portfolio_evaluation_df.loc["Annual Volatility"] = round(
        portret_df.std() * np.sqrt(252)
        ,2
        )

    # Calculate Sharpe ratio
    portfolio_evaluation_df.loc["Sharpe Ratio"] = round(
        portret_df.mean() * 252 / (
        portret_df.std() * np.sqrt(252))
        ,2
        )

    # Create a DataFrame that contains the Portfolio Daily Returns column
    sortino_ratio_df = portret_df.copy()

    # The Sortino ratio is reached by dividing the annualized return value
    # by the downside standard deviation value
    sortino_ratio = round(
            portret_df.mean() * 252 / (portret_df[portret_df < 0].std() * np.sqrt(252))
            ,2
            )

    # Add the Sortino ratio to the evaluation DataFrame
    portfolio_evaluation_df.loc["Sortino Ratio"] = round(sortino_ratio,2)    

    # Best and Worst streak
    from itertools import groupby

    # Best winning streak
    L = portret_df.copy()
    L[L > 0] = 1
    L[L < 0] = float("NaN")
    longest = max((list(g) for _, g in groupby(L)), key=len)
    portfolio_evaluation_df.loc["Best Winning Streak"] = len(longest)

    # Worst losing streak
    L = portret_df.copy()
    L[L < 0] = -1
    L[L > 0] = float("NaN")
    longest = max((list(g) for _, g in groupby(L)), key=len)
    portfolio_evaluation_df.loc["Worst Losing Streak"] = len(longest)
    
    print(portfolio_evaluation_df)

In [89]:
perf_eval(portret_df)

                    Backtest Results
Annualized Return               0.19
Cumulative Returns              0.94
Annual Volatility               0.25
Sharpe Ratio                    0.75
Sortino Ratio                   1.02
Best Winning Streak                9
Worst Losing Streak                8
